In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from torchtext.legacy import data
SEED= 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = "spacy",tokenizer_language = 'en_core_web_sm')
LABEL =data.LabelField(dtype = torch.float)

In [3]:
from torchtext.legacy import datasets
train_data , test_data = datasets.IMDB.splits(TEXT,LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 66.8MB/s]


In [4]:
print(f"Number of Examples in Training set :{len(train_data)}")
print(f"Number of examples in test set:{len(test_data)}")

Number of Examples in Training set :25000
Number of examples in test set:25000


In [5]:
print(vars(train_data[0]))

{'text': ['This', 'is', 'probably', 'the', 'fastest', '-', 'paced', 'and', 'most', 'action', '-', 'packed', 'of', 'the', 'German', 'Edgar', 'Wallace', '"', 'krimi', '"', 'series', ',', 'a', 'cross', 'between', 'the', 'Dr.', 'Mabuse', 'films', 'of', 'yore', 'and', '60', "'s", 'pop', 'thrillers', 'like', 'Batman', 'and', 'the', 'Man', 'from', 'UNCLE', '.', 'It', 'reintroduces', 'the', 'outrageous', 'villain', 'from', 'an', 'earlier', 'film', 'who', 'dons', 'a', 'stylish', 'monk', "'s", 'habit', 'and', 'breaks', 'the', 'necks', 'of', 'victims', 'with', 'the', 'curl', 'of', 'a', 'deadly', 'whip', '.', 'Set', 'at', 'a', 'posh', 'girls', "'", 'school', 'filled', 'with', 'lecherous', 'middle', '-', 'aged', 'professors', ',', 'and', 'with', 'the', 'cops', 'fondling', 'their', 'hot', '-', 'to', '-', 'trot', 'secretaries', 'at', 'every', 'opportunity', ',', 'it', 'certainly', 'is', 'a', 'throwback', 'to', 'those', 'wonderfully', 'politically', '-', 'incorrect', 'times', '.', 'There', "'s", 'a', 

In [6]:
import random 
train_data , validation_data = train_data.split(random_state = random.seed(SEED))

In [7]:
print(f"Number of validation examples: {len(train_data)}")
print(f"Number of examples in train_data :{len(test_data)}")

Number of validation examples: 17500
Number of examples in train_data :25000


In [8]:
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data,max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique words in the vocabulary:{len(TEXT.vocab)}")
print(f"The number of unique laabels is :{len(LABEL.vocab)}")

Unique words in the vocabulary:25002
The number of unique laabels is :2


In [10]:
print(TEXT.vocab.freqs.most_common(10))

[('the', 202223), (',', 192245), ('.', 165171), ('a', 108939), ('and', 108882), ('of', 100206), ('to', 93212), ('is', 76052), ('in', 61101), ('I', 53885)]


In [11]:
print(TEXT.vocab.itos[:10])
print(TEXT.vocab.stoi['and'])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']
6


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
BATCH_SIZE = 32
train_loader,valid_loader,test_loader = data.BucketIterator.splits((train_data,validation_data,test_data),batch_size = BATCH_SIZE,device = device)

In [14]:
import torch.nn as nn

In [15]:
class RNN(nn.Module):
    def __init__(self,input_dim,embedding_dim,hidden_dim,output_dim) :
        super().__init__()
        self.embedding = nn.Embedding(input_dim,embedding_dim)
        self.rnn = nn.RNN(embedding_dim,hidden_dim)
        self.fc = nn.Linear(hidden_dim,output_dim)
        
    def forward(self,text) :
        embedded = self.embedding(text)
        output,hidden = self.rnn(embedded)
        
        assert torch.equal(output[-1,:,:],hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

In [16]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM=1
model = RNN(INPUT_DIM,EMBEDDING_DIM,HIDDEN_DIM,OUTPUT_DIM)

In [17]:
def parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [18]:
params= parameters(model)
print(f"No of params in model is :{params}")

No of params in model is :2592105


In [19]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),lr = 1e-3)

In [20]:
criterion = nn.BCEWithLogitsLoss()

In [21]:
model = model.to(device)
criterion = criterion.to(device)


In [22]:
def accuracy(preds,y) :
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum()/len(correct)
    return acc

In [23]:
def train(model,iterator,optimizer,criterion) :
    epoch_loss=0
    epoch_acc=0
    model.train()
    
    for batch in iterator :
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions,batch.label)
        loss = criterion(predictions,batch.label)
        acc = accuracy(predictions,batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator),epoch_acc/len(iterator)   

In [24]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 24s
	Train Loss: 0.701 | Train Acc: 49.97%
	 Val. Loss: 0.694 |  Val. Acc: 50.73%
Epoch: 02 | Epoch Time: 0m 23s
	Train Loss: 0.700 | Train Acc: 49.56%
	 Val. Loss: 0.695 |  Val. Acc: 51.00%
Epoch: 03 | Epoch Time: 0m 23s
	Train Loss: 0.699 | Train Acc: 49.81%
	 Val. Loss: 0.695 |  Val. Acc: 51.12%
Epoch: 04 | Epoch Time: 0m 23s
	Train Loss: 0.699 | Train Acc: 49.42%
	 Val. Loss: 0.695 |  Val. Acc: 50.70%
Epoch: 05 | Epoch Time: 0m 23s
	Train Loss: 0.698 | Train Acc: 49.06%
	 Val. Loss: 0.694 |  Val. Acc: 51.14%


In [27]:


test_loss, test_acc = evaluate(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.692 | Test Acc: 47.11%
